In [227]:
import pandas as pd
import re
from sklearn.ensemble import IsolationForest  # Para detección de anomalías
import dash
from dash import dcc, html
import plotly.express as px  # Para visualización
from datetime import datetime, timedelta

In [228]:
#dir_path = "../process/"
#file= "external_test_data"# "train_data" #
# For folds
fold_N=0
dir_path = "../process/folds/"
file= f"val_fold_{fold_N}"#f"train_fold_{fold_N}"#
#logs_df = pd.read_csv(f"{dir_path}external_test_data.csv", encoding='utf-8')
logs_df = pd.read_csv(f"{dir_path}{file}.csv", encoding='utf-8')
print(set(logs_df["source"]))
print(set(logs_df["Level"]))
print(logs_df.shape)
logs_df.head()

{'Hadoop', 'BGL', 'Android', 'Apache', 'Zookeeper'}
{'Informativo', 'Error', 'Advertencia'}
(237, 6)


,Content,Level,datetime,content_length,source,Clean_Content
0,getRunningAppProcesses: caller 10113 does not ...,Advertencia,2024-03-17 16:14:54.045,6,Android,getrunningappprocesses caller does hold limiti...
1,Interrupting SendWorker,Advertencia,2015-07-29 19:34:08.526,2,Zookeeper,interrupting sendworker
2,"Connection broken for id 188978561024, my id =...",Advertencia,2015-07-29 19:22:15.277,5,Zookeeper,connection broken id id error
3,"Connection broken for id 188978561024, my id =...",Advertencia,2015-07-29 19:25:23.811,5,Zookeeper,connection broken id id error
4,"Connection broken for id 188978561024, my id =...",Advertencia,2015-07-29 19:33:11.012,5,Zookeeper,connection broken id id error


In [229]:
def get_severity(content):
    content_lower = content.lower()
    
    if any(x in content_lower for x in ['unreachable', 'workerenv error', 'goodbye', 'error contacting', 'forbidden',  'fatal', 'crash', 'unknown', 'empty', 'timeout', 'not responding', 'unexpected', 'error state']):
        return 'Crítico'
    elif any(x in content_lower for x in ['screen', 'broadcast', 'started', 'running', 'status', 'activated', 'cannot']):
        return 'Información'
    elif any(x in content_lower for x in [ 'dump','resume', 'connected', 'connection', 'ready', 'info', 'init', 'connect', 'message', 'connection', 'null', 'policy', 'ok', 'broken', 'terminating']):
        return 'Depuración'
    elif any(x in content_lower for x in ['denied', 'error state', 'overheat', 'failed', 'error', 'high', 'warn', 'alert', 'detected', 'exception',  'interrupted', 'interrupting', 'leaving', 'dropped', 'limiting', 'unable']):
        return 'Advertencia'
    else:
        return 'No relevante'

In [230]:
def get_origin(content):
    content_lower = content.lower()
    if any(x in content_lower for x in ['connect', 'disconnect','acquiring', 'detected', 'found', 'input', 'hover', 'change', 'submit', 'releasing', 'button', 'scroll', 'ready']):
        return 'Conectividad'
    if any(x in content_lower for x in ['boot', 'resume',  'stop', 'reboot', 'sleep', 'wake', 'standby', 'hibernat', 'workerenv']):
        return 'Aplicación'
    if any(x in content_lower for x in ['contacting', 'keepalive', 'info', 'slow', 'init', 'memory', 'forbidden', 'gc', 'block', 'fatal', 'denied', 'abort', 'load', 'leaving thread', 'usage', 'status', 'temperature']):
        return 'Sistema operativo'
    if any(x in content_lower for x in ['shutdown sock', 'renew', 'start service', 'waiting', 'interrupting', 'interrupted', 'caught end', 'sleep', 'wake', 'standby', 'hibernat']):
        return 'Gestión de energía'
    if any(x in content_lower for x in ['exception', 'fail', 'error', 'retry', 'illegal', 'forbidden', 'unexpected', 'null', 'unknown', 'crashed service', 'interrupted', 'interrupting', 'goodbye']):
        return 'Sistema de archivos'


    return 'Otro'


In [231]:
def get_event_type(content):
    content = content.lower()
    if any(x in content for x in ['boot',  'resume', 'start', 'stop', 'reboot', 'sleep', 'waiting message', 'message queue', 'wake', 'standby','hibernat', 'caller']):
        return 'Transicion'
    elif any(x in content for x in ['waiting', 'keepalive','info', 'slow', 'init', 'memory', 'battery', 'gc', 'block', 'fatal', 'denied', 'abort', 'load', 'leaving thread', 'usage', 'status', 'temperature']):
        return 'Monitoreo'
    elif any(x in content for x in ['down', 'shutdown', 'halt', 'terminate', 'terminated', 'terminating', 'killed', 'stopped', 'stop', 'exit', 'exited', 'aborted' ]):
        return 'Parada de procesos'
    elif any(x in content for x in ['scroll', 'start service', 'limiting', 'button', 'waiting', 'clicked', 'renew', 'connect', 'disconnect', 'detected', 'input', 'hover', 'change', 'submit', 'found']):
        return 'Eventos'
    elif any(x in content for x in ['exception', 'goodbye', 'fail', 'error', 'retry', 'crashed', 'illegal', 'forbidden', 'unexpected', 'null', 'unknown', 'crashed service', 'interrupted', 'interrupting' ]):
        return 'Errores'
    elif any(x in content for x in ['start service', 'terminating', 'stopped', 'exited', 'killed']):
        return 'Gestión de procesos'
    else:
        return 'Otro'

In [232]:
# Aplicar etiquetas
logs_df['Severidad'] = logs_df['Content'].apply(get_severity)
logs_df['Origen'] = logs_df['Content'].apply(get_origin)
logs_df['TipoEvento'] = logs_df['Content'].apply(get_event_type)

In [233]:
logs_df['datetime'] = pd.to_datetime(logs_df['datetime'], errors='coerce')
logs_df[logs_df["source"]=="Apache"].head()

,Content,Level,datetime,content_length,source,Clean_Content,Severidad,Origen,TipoEvento
79,mod_jk child workerEnv in error state 7,Error,2005-12-04 20:24:49,4,Apache,child workerenv error state,Crítico,Aplicación,Errores
80,mod_jk child workerEnv in error state 9,Error,2005-12-04 17:43:12,4,Apache,child workerenv error state,Crítico,Aplicación,Errores
81,mod_jk child workerEnv in error state 6,Error,2005-12-04 06:41:46,4,Apache,child workerenv error state,Crítico,Aplicación,Errores
82,mod_jk child workerEnv in error state 6,Error,2005-12-05 13:22:48,4,Apache,child workerenv error state,Crítico,Aplicación,Errores
83,mod_jk child workerEnv in error state 8,Error,2005-12-04 17:43:12,4,Apache,child workerenv error state,Crítico,Aplicación,Errores


In [234]:
import pandas as pd
from datetime import timedelta

# Asegura que 'datetime' sea tipo datetime
logs_df['datetime'] = pd.to_datetime(logs_df['datetime'], errors='coerce')

# Ordenar el DataFrame completo por datetime
logs_df = logs_df.sort_values(by='datetime')

# Nueva columna para las etiquetas de frecuencia
logs_df['Frecuencia'] = 'Sin etiquetar'

# Agrupar por 'Source' y 'Content' para evaluar eventos con contextos temporales distintos
grouped = logs_df.groupby(['source', 'Content'])

for (source, content), group in grouped:
    group = group.sort_values(by='datetime')
    count = len(group)

    if count == 1:
        freq_label = 'Único'
    else:
        time_deltas = group['datetime'].diff().dropna()
        avg_delta = time_deltas.mean()

        if avg_delta < timedelta(minutes=1) and count > 3:
            freq_label = 'Recurrente anómalo'
        elif timedelta(minutes=1) <= avg_delta <= timedelta(minutes=30) and time_deltas.std() < timedelta(seconds=5):
            freq_label = 'Recurrente esperado'
        elif any(x in content.lower() for x in ['doze mode active', 'no internet access', 'bluetooth on']):
            freq_label = 'Continuo'
        else:
            freq_label = 'Recurrente esperado'

    logs_df.loc[group.index, 'Frecuencia'] = freq_label


In [235]:

# Verificar el resultado
print(logs_df[['Content', 'Frecuencia']].drop_duplicates())

                                               Content           Frecuencia
101            mod_jk child workerEnv in error state 6  Recurrente esperado
107  [client 222.166.160.184] Directory index forbi...                Único
134            mod_jk child workerEnv in error state 7  Recurrente esperado
163  jk2_init() Found child 32478 in scoreboard slo...                Único
209  jk2_init() Found child 32549 in scoreboard slo...                Único
..                                                 ...                  ...
221                               generating core.2315                Único
222                              generating core.26835                Único
224                         iar 003a90fc dear 00b360e8                Único
234                         iar 001061b4 dear 0247017c                Único
236                              generating core.28329                Único

[140 rows x 2 columns]


In [236]:
from sklearn.ensemble import IsolationForest

# Feature engineering básico
logs_df['ContentLength'] = logs_df['Content'].apply(len)
logs_df['Hour'] = pd.to_datetime(logs_df['datetime'], format='%H:%M:%S.%f').dt.hour

# Variables numéricas para análisis
X = logs_df[['ContentLength', 'Hour']]
model = IsolationForest(contamination=0.05, random_state=42)
logs_df['Anomaly'] = model.fit_predict(X)
logs_df.head()

,Content,Level,datetime,content_length,source,Clean_Content,Severidad,Origen,TipoEvento,Frecuencia,ContentLength,Hour,Anomaly
101,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:52:52,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,-1
120,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:59:38,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,-1
148,mod_jk child workerEnv in error state 6,Error,2005-12-04 05:12:28,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,5.0,-1
107,[client 222.166.160.184] Directory index forbi...,Error,2005-12-04 05:15:09,8,Apache,client directory index forbidden rule var www ...,Crítico,Sistema operativo,Errores,Único,74,5.0,-1
134,mod_jk child workerEnv in error state 7,Error,2005-12-04 06:19:34,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,6.0,1


In [237]:
if fold_N!=None:
    dir_path="../process/folds_process/"

logs_df.to_csv(f"{dir_path}3_21{file}_heuristic_tagging.csv", index=False, encoding='utf-8')